<div style="border: 2px solid #FF6347; padding: 10px; border-radius: 5px; background-color: #FA8072; color: black; text-align: center;">
  <h1 style="margin: 0;">PULPO 🐙</h1>
</div>


### What Can I Do with PULPO? 🚀

With **PULPO**, you can:

- **(1) Optimize** (*minimize*) the environmental impacts of production systems that have **degrees of freedom**, such as technology 🔨 and regional 🌐 choices.
  
- **(2) Assess** production systems under various <span style="color:#FF6347">**constraints**</span>, including:
  - Production capacity
  - Raw material availability
  - Environmental regulations, etc.
- **(3) Reveal <span style="color:#4682B4">**trade**</span>-<span style="color:#32CD32">**offs**</span>** between different impact categories.

Example for different choices, constraints and trade-offs:
___
<div style="text-align: center;">
    <img src="figures/optimization.png" alt="System Boundaries of Rice Superstructure (Dark Theme)" width="550">
</div>

____
<div style="text-align: center;">
    <img src="figures/trade-off.png" alt="System Boundaries of Rice Superstructure (Dark Theme)" width="300">
</div>

____

But that's not all! You can also:
  
- **(4) Assess and optimize** the rollout of new technology across the **entire background system**

### Architecture and Theory

The illustration below shows the information flow within PULPO and the connectivity to other packages:

<div style="text-align: center;">
    <img src="figures/pulpo_infogram.png" alt="System Boundaries of Rice Superstructure (Dark Theme)" width="450">
</div>

<div style="background-color: #f9f9f9; padding: 20px; border: 2px solid #ccc; border-radius: 10px; font-family: Arial, sans-serif;">

  <h2 style="color: #0073e6;">Goal and Scope</h2>

  <div style="background-color: #f0f8ff; padding: 15px; margin: 15px 0; border-left: 4px solid #0073e6;">
    <h3 style="color: #0073e6;">📦 Functional Unit</h3>
    <p>The user specifies the functional unit via a Python dictionary, indexed by processes, to assign the final demand for each reference product. This is mandatory for every LCA/LCO and defines the quantity of products to be made available by the production system. Processes can be retrieved using the <code>retrieve_activities</code> function in the <code>PulpoOptimizer</code> class.</p>
  </div>

  <div style="background-color: #fffae6; padding: 15px; margin: 15px 0; border-left: 4px solid #ffcc00;">
    <h3 style="color: #ffcc00;">🎯 Objective Function</h3>
    <p>The objective function is the environmental impact category to minimize, set via the <code>methods</code> argument when creating a <code>PulpoOptimizer</code> object. Users can choose from LCIA methods bundled with ecoinvent or premise, or custom characterization factors such as the planetary boundary characterization factors (<a href="https://doi.org/10.1016/j.ecolind.2017.12.065">Ryberg et al. (2018)</a>; <a href="https://github.com/vtulus/AESAmethods">Brightway Implementation</a>). For example, specifying <code>('IPCC 2013', 'climate change', 'GWP 100a')</code>.</p>
  </div>

  <div style="background-color: #e6ffe6; padding: 15px; margin: 15px 0; border-left: 4px solid #228b22;">
    <h3 style="color: #228b22;">🔄 Possible Choices</h3>
    <p>These are sets of processes providing equivalent products/functions, defined by the user through a nested dictionary. This dictionary assigns labels to equivalent processes with capacity limits and is passed to the <code>PulpoOptimizer</code> via the <code>instantiate</code> function. For instance, if the functional unit is 1 kWh of electricity, processes may be subject to technology or regional choices.</p>
  </div>

  <div style="background-color: #fbe9e7; padding: 15px; margin: 15px 0; border-left: 4px solid #d32f2f;">
    <h3 style="color: #d32f2f;">⚙️ Additional Constraints</h3>
    <p>Users can set upper and lower bounds on processes through the <code>upper_limit</code> and <code>lower_limit</code> parameters in the <code>instantiate</code> function. Constraints on raw material availability, production capacities, and environmental flows can be enforced via <code>upper_imp_lim</code> and <code>upper_elem_lim</code> parameters. Further custom constraints can expand PULPO’s capabilities.</p>
  </div>

</div>

### Optimization Problem

The underlying optimization problem solved by PULPO can be summarized as follows:

$$
\text{min} \quad \left( \sum_h w_h \cdot z_h \right)
$$
Subject to:
<div style="color: orange;">
$$
\sum_j \left( \textcolor{red}{a_{ij}} \cdot s_j \right) = f_i + \text{slack}_i \quad \forall i 
$$
</div>
$$
s_j^{\text{low}} \leq s_j \leq s_j^{\text{high}} \quad \forall j 
$$
$$
0 \leq \text{slack}_i \leq \text{slack}_i^{\text{high}} \quad \forall i 
$$
<div style="color: dodgerblue;">
$$
z_h = \sum_e \sum_j \left( \textcolor{MediumOrchid}{q_{he}} \cdot \textcolor{MediumOrchid}{b_{ej}} \cdot s_j \right) \quad \forall h 
$$
</div>
$$
z_h \leq z_h^{\text{high}} \quad \forall h 
$$
$$
\sum_j \left( b_{ej} \cdot s_j \right) \leq b_e^{\text{high}} \quad \forall e 
$$

This notation is similar to the matrix notation of the TCM. Here, $i$ is the set of products, $j$ the set of processes, $e$ the set of environmental flows, and $h$ the set of impact categories. The coefficients $a$ are the entries of the technosphere matrix, $b$ the entries of the biosphere matrix, $q$ the characterization factors for the respective impact categories, and $f$ are the entries of the functional unit vector.

The $\textcolor{darkorange}{\text{orange}}$ equation represents the classic material balance, but the underlying $A$ matrix must be rectangular, rather than square, leading to degrees of freedom in the optimization problem. The $\textcolor{dodgerblue}{\text{blue}}$ equation corresponds to the impact calculation via the characterization factor matrix $Q$ and the biosphere matrix $B$.

The remainder of the elements in the problem formulation relate to constraints. For more details on the nomenclature, check the Supplementary Information of the [Paper](https://doi.org/10.1111/jiec.13561).
_____


### Bottom-Up vs. Top-Down Approach

The current form of square technosphere matrices is a consequence of the search for a unique solution to the inventory problem. See [Heijungs and Suh (2002)](https://link.springer.com/book/10.1007/978-94-015-9900-9) for an excellent overview of the computational structure of LCA, also commenting upon the topic of optimization (*Linear Programming*).

In this reference, the use of optimization to deal with multi-functional units that lead to rectangular matrices and non-unique solutions has been highlighted. The previous approach builds upon this idea, by constructing rectangular technology choice matrices in a <span style="color: dodgerblue; font-weight: bold;">bottom-up</span> fashion. The matrix is concatenated from various inventories, specifying identical products from distinct processes in the same row. See the [example data](data/rice_husk_example_TCM_data.xlsx) for the rice husk example from the following notebook. Compiling all of this data by hand in a spreadsheet ensures that the practitioner takes concious decisions about which data to include, but can be a limiting factor when moving to much more complex problems, as illustrated in the CCU case study in the [Paper](https://doi.org/10.1111/jiec.13561).

With PULPO we are proposing a <span style="color: orange; font-weight: bold;">top-down</span> approach, capitalizing on the already existing information in commercial and user LCIs. Departing from the square matrix, rows of identical products are summed up, leading to a technology (*or regional*) choice for the supply of this product in all other processes that utilize this product.

The Figures below illustrates the conceptual difference between the two approaches:

<span style="color: dodgerblue; font-weight: bold;">Bottom-Up</span> 🔼
<div style="text-align: center;">
    <img src="figures/bottom-up_LCO-TCM (Bottom-Up)_neutral.svg" alt="" width="600">
</div>

<span style="color: orange; font-weight: bold;">Top-Down</span> 🔽
<div style="text-align: center;">
    <img src="figures/bottom-up_LCO-PULPO (Top-Down)_neutral.svg" alt="" width="600">
</div>

__________________

### GitHub / Package 🔗

At the moment, PULPO is available in my personal [GitHub](https://github.com/flechtenberg/pulpo) repository. If the feedback is positive, we are considering deploying it as a plugin for the [activity-browser](https://github.com/LCA-ActivityBrowser/activity-browser).

The PULPO package is structured as follows:

- **[pulpo.py](https://github.com/flechtenberg/pulpo/blob/master/pulpo/pulpo.py)**: The main script containing the `PulpoOptimizer` class, which integrates all utility functions:
    - **[bw_parser](https://github.com/flechtenberg/pulpo/blob/master/pulpo/utils/bw_parser.py)**: Manages interactions between **Brightway** databases and PULPO.
    - **[converter](https://github.com/flechtenberg/pulpo/blob/master/pulpo/utils/converter.py)**: Processes raw LCI data and user inputs, converting them into the required format for the optimization problem.
    - **[optimizer](https://github.com/flechtenberg/pulpo/blob/master/pulpo/utils/optimizer.py)**: Formulates and solves the optimization problem using **Pyomo**.
    - **[saver](https://github.com/flechtenberg/pulpo/blob/master/pulpo/utils/saver.py)**: Provides functions to save results to `.xlsx` files or display them in the console using `pandas`.


The following parts of this workshop deal with various hands-on example case studies. Apart from these, there are various showcase notebooks in the [GitHub](https://github.com/flechtenberg/pulpo) repository:

- [hydrogen](https://github.com/flechtenberg/pulpo/blob/master/notebooks/hydrogen_showcase.ipynb)
- [electricity](https://github.com/flechtenberg/pulpo/blob/master/notebooks/energy_transition_showcase.ipynb) (energy transition)
- [plastic waste pyrolysis](https://github.com/flechtenberg/pulpo/blob/master/notebooks/plastic_showcase.ipynb)
